# ERK-KTR Full FOV Stimulation Pipeline

## System Init

### Load pymmcore and required python libraries

In [1]:
import os
os.environ['QT_LOGGING_RULES'] = '*.debug=false; *.warning=false' # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook

from fov import FOV
from useq import MDAEvent
import pandas as pd
import random
import napari
import pymmcore_plus
from napari_micromanager import MainWindow

from utils import create_folders
from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()

### Device Specific Init for Niesen Microscope

In [2]:
mmc.loadSystemConfiguration("E:\\MicroManagerConfigs\\Ti2CicercoConfig_w_DMD_22_w_ttl.cfg")
mmc.mda.engine.use_hardware_sequencing = False


### The following libraries are used to wake up the Lumencor laser
import requests
import threading
import time

class WakeUpLaser:
    def __init__(self, lumencore_ip="192.168.201.200"): 
        self.ip = lumencore_ip
        self.last_wakeup = 0
        self.is_running = False
        
    def wakeup_laser(self):
        url = f"http://{self.ip}/service/?command=WAKEUP"
        requests.get(url)
    
    def run(self, wait_for_warmup=False):
        self.is_running = True
        self.thread = threading.Thread(target=self._keep_alive)
        self.thread.start()
        if wait_for_warmup:
            time.sleep(15)

    def _keep_alive(self):
        while self.is_running:
            if time.time() - self.last_wakeup > 60:
                self.wakeup_laser()
                self.last_wakeup = time.time()
            time.sleep(3)
    def stop(self):
        self.is_running = False
        self.thread.join()
        
wl = WakeUpLaser()
wl.wakeup_laser()

slm_dev = mmc.getSLMDevice()
slm_width = mmc.getSLMWidth(slm_dev)
slm_height = mmc.getSLMHeight(slm_dev)

event_slm_on = MDAEvent(slm_image=SLMImage(data=True))
mmc.mda.run([event_slm_on])# to only have fov of DMD 
mmc.setROI(150, 150, 1900, 1900)

DMD_CHANNEL_GROUP = "WF_DMD"
DMD_CALIBRATION_PROFILE = {"channel_group": "WF_DMD", "channel_config": "CyanStim", "device_name": "LedDMD", "property_name": "Cyan_Level", "power": 100}
mmc.setChannelGroup(channelGroup=DMD_CHANNEL_GROUP)

[01/17/25 19:45:40] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=950052;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=410717;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#328\328]8;;\

                    INFO     slm_image=SLMImage()                                                    ]8;id=507964;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=682314;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\

                    INFO     MDA Finished: GeneratorMDASequence()                                    ]8;id=270403;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=539049;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#415\415]8;;\

## GUI - Napari Micromanager

### Load GUI

In [3]:
### Base GUI ###
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [4]:
### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget
mdawidget = MDAWidget(mmcore = mmc)
viewer.window.add_dock_widget(mdawidget)

### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [10]:
### Break connection
# mm_wdg._core_link.cleanup()

In [ ]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink
mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## Create a DF with all planned acquisitions and stimulations

### Settings for Experiment

In [11]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

base_path = "C:\\test"
experiment_name = "exp_23"
path  = os.path.join(base_path, experiment_name)

create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

time_between_frames = 30 #time in seconds between frames
time_per_fov = 15 #time in seconds per fov

timesteps = range(3)  
channels = ['Red', 'Green'] #channel for segmentation first
channels_exposure = [500, 500]

# take values from UI, if loaded
intensity_red_laser = mmc.getProperty("Laser", "RED_Intensity")
intensity_green_laser = mmc.getProperty("Laser", "GREEN_Intensity")

if intensity_red_laser != str(0) and intensity_green_laser != str(0):
    channels_power = [intensity_red_laser, intensity_green_laser]
else:
    channels_power = [50, 50]
    for channel, power in zip(channels, channels_power):
        mmc.setProperty("Laser", f"{channel.upper()}_Intensity", power)


cell_lines = ["optoFGFR1"] ## Spalten bei Wellplate

stim_exposures = [100, 200] # list of possible exposures in ms
stim_timesteps = [[1]]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]
stim_profiles = [{"device_name": "LedDMD", "property_name": "Cyan_Level", "power": 10, "channel": "CyanStim"}]
stim_treatment = [{"stim_property": "global", "stim_profile": stim_profile, "stim_exposure": stim_exposure, "stim_timestep": stim_timestep} 
                  for stim_profile in stim_profiles for stim_exposure in stim_exposures for stim_timestep in stim_timesteps]
random.shuffle(stim_treatment)

# if defining individual fovs, else these values are ignored: 
n_fovs_per_cell_line = 2 ## change this variable to the amount of fovs that you have per cell line. If only one cell line is set, this value will 
                        # automatically set to total amount of fovs. If you are working will wellplate, this value will be ignored, as each columns
                        # will be an entry in the cell lines list. 

data_mda_fovs = None
stim_treatment

Directory C:\test\exp_23\stim already exists
Directory C:\test\exp_23\raw already exists
Directory C:\test\exp_23\labels already exists
Directory C:\test\exp_23\stim_mask already exists
Directory C:\test\exp_23\tracks already exists
Directory C:\test\exp_23\labels_rings already exists
Directory C:\test\exp_23\particles already exists


[{'stim_property': 'global',
  'stim_profile': {'device_name': 'LedDMD',
   'property_name': 'Cyan_Level',
   'power': 10,
   'channel': 'CyanStim'},
  'stim_exposure': 200,
  'stim_timestep': [1]},
 {'stim_property': 'global',
  'stim_profile': {'device_name': 'LedDMD',
   'property_name': 'Cyan_Level',
   'power': 10,
   'channel': 'CyanStim'},
  'stim_exposure': 100,
  'stim_timestep': [1]}]

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [ ]:
import json
file = os.path.join(path, "fovs.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)

Only select one of the following two code blocks. If you autogenerated FOVs using the wellplate option of the MDA widget, then use the first code block, else the second. 

#### MDA Widget was used in wellplate mode with autogenerated FOVs

In [ ]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value()
n_fovs = len(data_mda_fovs.stage_positions)
pts_per_well = data_mda_fovs.stage_positions.well_points_plan.num_points
n_wells = n_fovs // pts_per_well

used_well_columns = []

for i, row in enumerate(data_mda_fovs.stage_positions):
    well_column = int(row.name.split('_')[0][1:])
    used_well_columns.append(well_column)

unique_well_columns = list(set(used_well_columns))
unique_well_column_translation = {col: idx for idx, col in enumerate(unique_well_columns)}
  

for i, row in enumerate(data_mda_fovs.stage_positions):
    well_row = row.name.split('_')[0][0]
    well_id = i%pts_per_well
    if len(cell_lines) == 1:
        cell_line = cell_lines[0]
    else: 
        cell_line = cell_lines[unique_well_column_translation[well_column]]
    fov = FOV(pos=(row.x, row.y),
              index=i,
              name=row.name,
              path=path,
              metadata={"well_column": well_column, "well_row": well_row, "well_id": well_id, "cell_line": cell_line},
              treatment=stim_treatment[i%len(stim_treatment)],
              )
    fovs.append(fov)

#### FOVs were manually selected using MDA widget

In [13]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value().stage_positions
n_fovs = len(data_mda_fovs)
n_stim_treatments = len(stim_treatment)
n_fovs_per_stim_condition = n_fovs // n_stim_treatments
stim_treatment_tot = stim_treatment * n_fovs_per_stim_condition
if n_fovs % n_stim_treatments != 0:
    print(f"Warning: Not equal number of fovs per stim condition. {n_fovs % n_stim_treatments} fovs will have repeated treatment")
    stim_treatment_tot.extend(stim_treatment[:n_fovs % n_stim_treatments])

if len(cell_lines) == 1: 
    n_fovs_per_cell_line = n_fovs
 
random.shuffle(stim_treatment_tot)

for i, row in enumerate(data_mda_fovs):
    print
    row = dict(row)
    if len(cell_lines) == 1:
        cell_line = cell_lines[0]
    else:
        cell_line = cell_lines[i % n_fovs_per_cell_line]
    fov = FOV(pos=(row["x"], row["y"]),
              index=i,
              name=str(i),
              path=path,
              metadata={"cell_line": cell_line},
              treatment=stim_treatment_tot[i],
              )
    fovs.append(fov)

print(f"Doing {n_fovs_per_stim_condition} replicates per stim condition")

Doing 0 replicates per stim condition


### Use FOVs to generate dataframe for acquisition

In [14]:
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0

dfs = []
for fov in fovs:
    fov_group = fov.index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)

    for timestep in timesteps:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'name':fov.name,
                    'timestep': timestep,
                    'time': start_time + timestep*time_between_frames,
                    'treatment': fov.treatment,
                    'metadata': fov.metadata,
                    'stim': timestep in fov.treatment['stim_timestep'], # not really important, only for visualisation
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_power': channels_power,
                    'stim_exposure' : fov.treatment['stim_exposure'], # not really important, only for visualisation
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    }
        dfs.append(new_row)

df_acquire = pd.DataFrame(dfs)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)
df_acquire = df_acquire.sort_values(by=['time', 'fov'])
print(f"Total Experiment Time: {df_acquire['time'].max()}s")
df_acquire


Total Experiment Time: 60s


,fov_object,fov,name,timestep,time,treatment,metadata,stim,channels,channels_exposure,channel_power,stim_exposure,fname
0,<fov.FOV object at 0x0000023ED6A35FF0>,0,0,0,0,"{'stim_property': 'global', 'stim_profile': {'...",{'cell_line': 'optoFGFR1'},False,"[Red, Green]","[500, 500]","[50, 50]",200,000_00000
1,<fov.FOV object at 0x0000023ED6A35FF0>,0,0,1,30,"{'stim_property': 'global', 'stim_profile': {'...",{'cell_line': 'optoFGFR1'},True,"[Red, Green]","[500, 500]","[50, 50]",200,000_00001
2,<fov.FOV object at 0x0000023ED6A35FF0>,0,0,2,60,"{'stim_property': 'global', 'stim_profile': {'...",{'cell_line': 'optoFGFR1'},False,"[Red, Green]","[500, 500]","[50, 50]",200,000_00002


## Run experiment

In [15]:
%load_ext autoreload
%autoreload 2

from add_frame import ImageProcessingPipeline
from segmentation_stardist import SegmentatorStardist
from stimulation import StimWholeFOV    
from controller import Controller, Analyzer
from tracking_trackpy import TrackerTrackpy
from dmd import DMD
from queue import Queue

try: 
    mm_wdg._core_link.cleanup()
except:
    pass

segmentator = SegmentatorStardist()
stimulator = StimWholeFOV()
tracker = TrackerTrackpy()
dmd = DMD(mmc, DMD_CALIBRATION_PROFILE)

pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)
queue = Queue()
controller = Controller(analyzer, mmc, queue, dmd)
wl.run(wait_for_warmup=True)
controller.run(df_acquire)
wl.stop()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


[01/17/25 19:49:47] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=355358;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=519336;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#328\328]8;;\

                    INFO     index={'t': 0, 'c': 0, 'p': 0} channel=Channel(config='Red',            ]8;id=371247;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=350586;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=500.0 min_start_time=0.0 x_pos=-19064.5                      
                             y_pos=4063.7 metadata={'fov_object': <fov.FOV object at                               
                             0x0000023ED6A35FF0>, 'fov': 0, 'name': '0', 'timestep': 0, 'time': 0,                 
                             'treatment': {'stim_property': 'global', 'stim_profile':                              
                             {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10,                 
                             'channel': 'CyanStim'}, 'stim_exposure': 200, 'stim_timestep': [1]},                  
                             'metadata': {'cell_line': 'optoFGFR1'}, 'stim': False, 'channels':                    
                             ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power':                   
                             ['50', '50'], 'stim_exposure': 200, 'fname': '000_00000', 'img_type':                 
                             <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}                        

[01/17/25 19:49:49] INFO     index={'t': 0, 'c': 1, 'p': 0} channel=Channel(config='Green',          ]8;id=599896;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=138196;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=500.0 min_start_time=0.0 x_pos=-19064.5                      
                             y_pos=4063.7 metadata={'fov_object': <fov.FOV object at                               
                             0x0000023ED6A35FF0>, 'fov': 0, 'name': '0', 'timestep': 0, 'time': 0,                 
                             'treatment': {'stim_property': 'global', 'stim_profile':                              
                             {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10,                 
                             'channel': 'CyanStim'}, 'stim_exposure': 200, 'stim_timestep': [1]},                  
                             'metadata': {'cell_line': 'optoFGFR1'}, 'stim': False, 'channels':                    
                             ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power':                   
                             ['50', '50'], 'stim_exposure': 200, 'fname': '000_00000', 'img_type':                 
                             <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Green'}                       

functional.py (237): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 1904, 1904, 1))


...

[01/17/25 19:50:18] INFO     index={'t': 1, 'c': 1, 'p': 0} channel=Channel(config='Green',          ]8;id=973328;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=930580;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=500.0 min_start_time=30.0 x_pos=-19064.5                     
                             y_pos=4063.7 metadata={'fov_object': <fov.FOV object at                               
                             0x0000023ED6A35FF0>, 'fov': 0, 'name': '0', 'timestep': 1, 'time': 30,                
                             'treatment': {'stim_property': 'global', 'stim_profile':                              
                             {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10,                 
                             'channel': 'CyanStim'}, 'stim_exposure': 200, 'stim_timestep': [1]},                  
                             'metadata': {'cell_line': 'optoFGFR1'}, 'stim': True, 'channels':                     
                             ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power':                   
                             ['50', '50'], 'stim_exposure': 200, 'fname': '000_00001', 'img_type':                 
                             <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Green'}                       

[01/17/25 19:50:19] INFO     index={'t': 1, 'p': 0} channel=Channel(config='CyanStim',               ]8;id=267539;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=385731;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=200.0 min_start_time=30.0 x_pos=-19064.5                     
                             y_pos=4063.7 slm_image=SLMImage()                                                     
                             properties=[PropertyTuple(device_name='LedDMD',                                       
                             property_name='Cyan_Level', property_value=10)] metadata={'fov_object':               
                             <fov.FOV object at 0x0000023ED6A35FF0>, 'fov': 0, 'name': '0',                        
                             'timestep': 1, 'time': 30, 'treatment': {'stim_property': 'global',                   
                             'stim_profile': {'device_name': 'LedDMD', 'property_name':                            
                             'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure':                   
                             200, 'stim_timestep': [1]}, 'metadata': {'cell_line': 'optoFGFR1'},                   
                             'stim': True, 'channels': ['Red', 'Green'], 'channels_exposure': [500,                
                             500], 'channel_power': ['50', '50'], 'stim_exposure': 200, 'fname':                   
                             '000_00001', 'img_type': <ImgType.IMG_STIM: 2>, 'last_channel': True,                 
                             'channel': 'CyanStim'}                                                                

[01/17/25 19:50:47] INFO     index={'t': 2, 'c': 0, 'p': 0} channel=Channel(config='Red',            ]8;id=157229;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=368663;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=500.0 min_start_time=60.0 x_pos=-19064.5                     
                             y_pos=4063.7 metadata={'fov_object': <fov.FOV object at                               
                             0x0000023ED6A35FF0>, 'fov': 0, 'name': '0', 'timestep': 2, 'time': 60,                
                             'treatment': {'stim_property': 'global', 'stim_profile':                              
                             {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10,                 
                             'channel': 'CyanStim'}, 'stim_exposure': 200, 'stim_timestep': [1]},                  
                             'metadata': {'cell_line': 'optoFGFR1'}, 'stim': False, 'channels':                    
                             ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power':                   
                             ['50', '50'], 'stim_exposure': 200, 'fname': '000_00002', 'img_type':                 
                             <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}                        

[01/17/25 19:50:48] INFO     index={'t': 2, 'c': 1, 'p': 0} channel=Channel(config='Green',          ]8;id=510948;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=194391;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             group='WF_DMD') exposure=500.0 min_start_time=60.0 x_pos=-19064.5                     
                             y_pos=4063.7 metadata={'fov_object': <fov.FOV object at                               
                             0x0000023ED6A35FF0>, 'fov': 0, 'name': '0', 'timestep': 2, 'time': 60,                
                             'treatment': {'stim_property': 'global', 'stim_profile':                              
                             {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10,                 
                             'channel': 'CyanStim'}, 'stim_exposure': 200, 'stim_timestep': [1]},                  
                             'metadata': {'cell_line': 'optoFGFR1'}, 'stim': False, 'channels':                    
                             ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power':                   
                             ['50', '50'], 'stim_exposure': 200, 'fname': '000_00002', 'img_type':                 
                             <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Green'}                       

                    INFO     MDA Finished: GeneratorMDASequence()                                    ]8;id=76143;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=295025;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#415\415]8;;\